In [1]:
import re
import json
import hashlib
import numpy as np
import pandas as pd
from itertools import permutations

# Day 1

In [2]:
with open("./data/adventofcode.com_2015_day_1_input") as fid:
    content = fid.read()

In [3]:
level = 0
for c in content:
    if c == "(":
        level += 1
    elif c == ")":
        level -= 1
    else:
        raise ValueError(c)

In [4]:
level

232

In [5]:
level = 0
for idx, c in enumerate(content):
    if c == "(":
        level += 1
    elif c == ")":
        level -= 1
    else:
        raise ValueError(c)
    if level == -1:
        print(idx)
        break

1782


# Day 2

In [81]:
df = pd.read_csv("./data/adventofcode.com_2015_day_2_input", sep="x", names=["l", "w", "h"])

In [82]:
df["lw"] = df["l"] * df["w"]
df["wh"] = df["w"] * df["h"]
df["hl"] = df["h"] * df["l"]
df["small"] = [min(b) for a, b in df[["lw", "wh", "hl"]].iterrows()]
df["total"] = 2 * df["lw"] + 2 * df["wh"] + 2 * df["hl"] + df["small"]
df["total"].sum()

1598415

In [83]:
df["perimeter_small"] = [2 * sum(sorted(b)[:2]) for a, b in df[["l", "w", "h"]].iterrows()]
df["wrap"] =  df["h"] * df["w"] * df["l"] + df["perimeter_small"]
df["wrap"].sum()

3812909

# Day 3

In [42]:
with open("./data/adventofcode.com_2015_day_3_input.txt") as fid:
    content = fid.read()

In [51]:
nh = len(content)
houses = np.zeros((nh * 2 + 1, nh * 2 + 1), dtype=np.int16)

i = nh + 1
j = nh + 1
houses[i, j] = 1
for c in content:
    if c == "^":
        j += 1
    elif c == "v":
        j -= 1
    elif c == ">":
        i += 1
    elif c == "<":
        i -= 1
    houses[i, j] += 1
    

In [56]:
np.sum(houses > 0)

2572

In [59]:
houses = np.zeros((nh * 2 + 1, nh * 2 + 1), dtype=np.int16)

i = nh + 1
j = nh + 1
houses[i, j] = 2
for c in content[::2]:
    if c == "^":
        j += 1
    elif c == "v":
        j -= 1
    elif c == ">":
        i += 1
    elif c == "<":
        i -= 1
    houses[i, j] += 1

i = nh + 1
j = nh + 1
for c in content[1::2]:
    if c == "^":
        j += 1
    elif c == "v":
        j -= 1
    elif c == ">":
        i += 1
    elif c == "<":
        i -= 1
    houses[i, j] += 1

In [60]:
np.sum(houses > 0)

2631

# Day 4

In [75]:
for num in range(1, 10_000_000):
    result = hashlib.md5(f'iwrupvqb{num}'.encode())
    if result.hexdigest().startswith("00000"):
        print(num)
        break

9958218


# Day 5

In [87]:
from collections import Counter

In [101]:
with open("data/adventofcode.com_2015_day_5_input.txt") as fid:
    content = fid.read().splitlines()

In [102]:
good = 0
bad = 0
for line in content:
    test = len(re.findall("(\w)\\1", line)) > 0 
    test &= sum([Counter(line)[vow] for vow in "aeiou"]) >= 3
    test &= not any([i in line for i in ["ab", "cd", "pq", "xy"]])
    if test:
        good += 1
    else:
        bad += 1

In [103]:
good

258

In [121]:
line = "qjhvhtzxzqqjkmpb"

In [131]:
count = sum(
      1 for s in content
      if len(re.findall(r"([a-z]{2}).*\1", s))
      and re.findall(r"([a-z]).\1", s)
 )
print(count)

53


# Day 6

In [160]:
with open("data/adventofcode.com_2015_day_6_input.txt") as fid:
    content = fid.read().splitlines()

In [165]:
def get_coord(line):
    start_str, end_str = re.findall("[0-9]+,[0-9]+", line)
    i, j = start_str.split(",")
    k, l = end_str.split(",")
    return int(i), int(j), int(k) + 1, int(l) + 1

In [166]:
grid = np.zeros((1000, 1000), dtype=np.int16)
for line in content:
    i, j, k, l = get_coord(line)
    if line.startswith("turn on"):
        grid[slice(i, k), slice(j, l)] = 1
    elif line.startswith("turn off"):
        grid[slice(i, k), slice(j, l)] = 0
    elif line.startswith("toggle"):
        grid[slice(i, k), slice(j, l)] += 1
        grid[slice(i, k), slice(j, l)] = grid[slice(i, k), slice(j, l)] % 2

In [175]:
grid.sum()

15343601

In [172]:
grid = np.zeros((1000, 1000), dtype=np.int16)
for line in content:
    i, j, k, l = get_coord(line)
    if line.startswith("turn on"):
        grid[slice(i, k), slice(j, l)] += 1
    elif line.startswith("turn off"):
        grid[slice(i, k), slice(j, l)] -= 1
        grid[grid < 0] = 0
    elif line.startswith("toggle"):
        grid[slice(i, k), slice(j, l)] += 2

In [173]:
grid.sum()

15343601

# Day 7

In [193]:
with open("data/adventofcode.com_2015_day_7_input.txt") as fid:
    commands = fid.read().splitlines()

In [194]:
calc = dict()
results = dict()

for command in commands:
    (ops, res) = command.split('->')
    calc[res.strip()] = ops.strip().split(' ')

def calculate(name):
    try:
        return int(name)
    except ValueError:
        pass

    if name not in results:
        ops = calc[name]
        if len(ops) == 1:
            res = calculate(ops[0])
        else:
            op = ops[-2]
            if op == 'AND':
                res = calculate(ops[0]) & calculate(ops[2])
            elif op == 'OR':
                res = calculate(ops[0]) | calculate(ops[2])
            elif op == 'NOT':
                res = ~calculate(ops[1]) & 0xffff
            elif op == 'RSHIFT':
                res = calculate(ops[0]) >> calculate(ops[2])
            elif op == 'LSHIFT':
                res = calculate(ops[0]) << calculate(ops[2])
        results[name] = res
    return results[name]

print("a: %d" % calculate('a'))

a: 14710


# Day 8

In [6]:
print(sum(len(s[:-1]) - len(eval(s)) for s in open("data/adventofcode.com_2015_day_8_input.txt")))

1333


In [7]:
print(sum(2+s.count('\\')+s.count('"') for s in open("data/adventofcode.com_2015_day_8_input.txt")))

2046


# Day 9

In [26]:
content = """Faerun to Tristram = 65
Faerun to Tambi = 129
Faerun to Norrath = 144
Faerun to Snowdin = 71
Faerun to Straylight = 137
Faerun to AlphaCentauri = 3
Faerun to Arbre = 149
Tristram to Tambi = 63
Tristram to Norrath = 4
Tristram to Snowdin = 105
Tristram to Straylight = 125
Tristram to AlphaCentauri = 55
Tristram to Arbre = 14
Tambi to Norrath = 68
Tambi to Snowdin = 52
Tambi to Straylight = 65
Tambi to AlphaCentauri = 22
Tambi to Arbre = 143
Norrath to Snowdin = 8
Norrath to Straylight = 23
Norrath to AlphaCentauri = 136
Norrath to Arbre = 115
Snowdin to Straylight = 101
Snowdin to AlphaCentauri = 84
Snowdin to Arbre = 96
Straylight to AlphaCentauri = 107
Straylight to Arbre = 14
AlphaCentauri to Arbre = 46
""".splitlines()

In [27]:
data = {}
cities = []
for line in content:
    nar = re.findall("(\w+) to (\w+) = ([0-9]+)", line)[0]
    cities.append(nar[0])
    cities.append(nar[1])
    if nar[0] in data.keys():
        data[nar[0]] = {nar[1]: int(nar[2]), **data[nar[0]]}
    else:
        data[nar[0]] = {nar[1]: int(nar[2])}

In [28]:
ci = np.unique(cities)
nir = [*permutations(np.arange(len(ci)))]

In [29]:
def get_iter(arr, v):
    if len(arr) == 1:
        return arr[0]
    else:
        return [a for a in arr if a != v]

In [30]:
def get_dist(city1, city2):
    global data
    try:
        d = data[city1][city2]
    except KeyError:
        d = data[city2][city1]
    return d

In [31]:
dist = np.zeros((len(nir)))
for idx, indices in enumerate(nir):
    for i0, i1 in zip(indices[:-1], indices[1:]):
        dist[idx] += get_dist(ci[i0], ci[i1])

In [32]:
dist.max()

909.0

# Day 10

In [46]:
line = "3113322113"
cnt = 0
while cnt < 50 :
    newline = ""
    count = 1
    similar = False
    for i in range(len(line) - 1):
        if line[i] == line[i + 1]:
            count += 1
            similar = True
        else:
            newline += f"{count}{line[i]}"
            count = 1
            similar = False
    
    if similar:
#         count += 1
        newline += f"{count}{line[-1]}"
    else:
        newline += f"1{line[-1]}"
        
    cnt += 1
    line = newline
    print(len(line))
            

12
18
22
28
38
48
60
76
100
118
146
192
246
328
428
562
732
970
1250
1622
2128
2754
3592
4728
6162
8072
10504
13736
17874
23248
30322
39392
51438
67070
87402
114056
148740
193828
252594
329356
429204
559092
729412
950310
1239364
1615860
2106324
2746068
3579328
4666278


# Day 11

In [88]:
def check_seq(t):
    idx = 0
    count = 0
    rslt = False
    while idx < len(t) - 1:
        if t[idx] == t[idx + 1] - 1:
            count += 1
        else:
            count = 0
        if count == 2:
            rslt = True
            break

        idx += 1
    return rslt

In [86]:
def increment(t, idx):
    last = ord("z")
    start = ord("a")
    
    t[idx] += 1
    if t[idx] > last:
        t[idx] = start
        t = increment(t, idx - 1)
    
    return t

In [128]:
def detect_consecutive_pairs(lst):
    pairs = []
    triplet = False
    for i in range(len(lst) - 1):
        if triplet:
            triplet = False
            continue
    
        # If the current element is equal to the next element
        if lst[i] == lst[i + 1]:
            if i < len(lst) - 2:
                if lst[i + 1] == lst[i + 2]:
                    triplet = True
                    continue

            pairs.append((lst[i], lst[i + 1]))

    return len(set([p[0] for p in pairs])) > 1

In [135]:
pw = "hepxxzaa"
t = np.array([ord(c) for c in pw])
forbid = [ord("i"), ord("l"), ord("o")]
last = ord("z")
start = ord("a")

while True:
    if any([a in forbid for a in t]):
        idx = np.where((t == ord("i")) | (t == ord("l")) | (t == ord("o")))[0].min()
        t[idx] += 1
        t[idx + 1 :] = start
        continue
    if not check_seq(t):
        t = increment(t, 7)
        continue
    if not detect_consecutive_pairs(t):
        t = increment(t, 7)
        continue
    
    pw = [chr(i) for i in t]
    print("".join(pw))
    break

heqaabcc


In [134]:
pw = "hepxxyzz"
t = np.array([ord(c) for c in pw])
t = increment(t, 7)
pw = [chr(i) for i in t]
print("".join(pw))

hepxxzaa


# Day 12

In [136]:
with open("./data/adventofcode.com_2015_day_12_input.txt") as fid:
    content = fid.read()

In [158]:
nb = [int(i) for i in re.findall("-?\\d+",content)]
print(sum(nb))

191164


In [155]:
data = json.loads(content)

In [162]:
def n(j):
    if type(j) == int:
        return j
    if type(j) == list:
        return sum([n(j) for j in j])
    if type(j) != dict:
        return 0
    if 'red' in j.values():
        return 0
    return n(list(j.values()))

# print(n(loads(input())))

In [163]:
n(data)

87842

# Day 13

In [15]:
with open("./data/adventofcode.com_2015_day_13_input.txt") as fid:
    txt = fid.read()

In [16]:
txt2 = txt.replace("lose ", "-")
data = {}
people = []
for line in txt2.splitlines():
    nar = re.findall("(-?\\d+)", line)[0]
    p0 = line.split(" ")[0]
    p1 = line.split(" ")[-1].rstrip(".")
    if p0 not in people:
        people.append(p0)
    if p1 not in people:
        people.append(p1)
        
    if p0 in data.keys():
        data[p0] = {p1: int(nar), **data[p0]}
    else:
        data[p0] = {p1: int(nar)}

In [17]:
nir = [*permutations(np.arange(len(people)))]

In [19]:
def get_happy(p0, p1):
    global data
    d = data[p0][p1]
    return d

In [20]:
happy = np.zeros((len(nir)), dtype=int)
for cnt, idx in enumerate(nir):
    tmp = [*idx]
    tmp.append(idx[0])
    happy[cnt] = sum([get_happy(people[a], people[b]) for a, b in zip(tmp[:-1], tmp[1:])])
    happy[cnt] += sum([get_happy(people[a], people[b]) for a, b in zip(tmp[::-1][:-1], tmp[::-1][1:])])

In [21]:
happy.max()

664

In [22]:
for k, v in data.items():
    data[k] = {"me": 0, **v}
    
data["me"] = {k: 0 for k in people}
people.append("me")

In [23]:
nir = [*permutations(np.arange(len(people)))]

In [24]:
happy = np.zeros((len(nir)), dtype=int)
for cnt, idx in enumerate(nir):
    tmp = [*idx]
    tmp.append(idx[0])
    happy[cnt] = sum([get_happy(people[a], people[b]) for a, b in zip(tmp[:-1], tmp[1:])])
    happy[cnt] += sum([get_happy(people[a], people[b]) for a, b in zip(tmp[::-1][:-1], tmp[::-1][1:])])

In [25]:
happy.max()

640

# Day 14

In [249]:
content = """Dancer can fly 27 km/s for 5 seconds, but then must rest for 132 seconds.
Cupid can fly 22 km/s for 2 seconds, but then must rest for 41 seconds.
Rudolph can fly 11 km/s for 5 seconds, but then must rest for 48 seconds.
Donner can fly 28 km/s for 5 seconds, but then must rest for 134 seconds.
Dasher can fly 4 km/s for 16 seconds, but then must rest for 55 seconds.
Blitzen can fly 14 km/s for 3 seconds, but then must rest for 38 seconds.
Prancer can fly 3 km/s for 21 seconds, but then must rest for 40 seconds.
Comet can fly 18 km/s for 6 seconds, but then must rest for 103 seconds.
Vixen can fly 18 km/s for 5 seconds, but then must rest for 84 seconds."""

In [250]:
names = [n.split(" ")[0] for n in content.splitlines()]

In [254]:
speed = []
time_moving = []
time_resting = []
for n in content.splitlines():
    a, b, c = re.findall("\\d+", n)
    speed.append(int(a))
    time_moving.append(int(b))
    time_resting.append(int(c))

In [288]:
class Reindeer:
    def __init__(self, name, speed, move, rest):
        self.name = name
        self.speed = speed
        self.time_moving = move
        self.time_resting = rest
        self.distance = 0
        self.state_time = 0
        self.clock = 0
        self.is_resting = False
        self.points = 0
        
    def do_increment(self):
        self.state_time += 1
        self.clock += 1
        
        if self.is_resting:
            if self.state_time == self.time_resting:
                self.fly()
        else:
            self.distance += self.speed
            if self.state_time == self.time_moving:
                self.stop()
                
    def fly(self):
        self.state_time = 0
        self.is_resting = False
        
    def stop(self):
        self.state_time = 0
        self.is_resting = True
        
    def get_distance(self):
        return self.distance
    
    def reward(self):
        self.points += 1
    

In [273]:
reins = [None] * len(names)
for i in range(len(names)):
    reins[i] = Reindeer(
        names[i],
        speed[i],
        time_moving[i],
        time_resting[i]
    )

In [276]:
for i in range(2503):
    [r.do_increment() for r in reins]
    
for i in range(len(names)):
    print(f"{reins[i].name}: {reins[i].get_distance()} km")

Dancer: 2565 km
Cupid: 2596 km
Rudolph: 2640 km
Donner: 2548 km
Dasher: 2304 km
Blitzen: 2590 km
Prancer: 2589 km
Comet: 2484 km
Vixen: 2610 km


In [277]:
dist = [r.get_distance() for r in reins]

In [289]:
reins = [None] * len(names)
for i in range(len(names)):
    reins[i] = Reindeer(
        names[i],
        speed[i],
        time_moving[i],
        time_resting[i]
    )

In [290]:
for i in range(2503):
    [r.do_increment() for r in reins]
    dist = np.array([r.get_distance() for r in reins])
    leaders = np.where(dist.max() == dist)[0]
    for j in leaders:
        reins[j].reward()


In [292]:
max([r.points for r in reins])

1102

# Day 15

C++ code in Ubuntu

# Day 16

In [80]:
with open("data/adventofcode.com_2015_day_16_input") as fid:
    content = fid.read().splitlines()

In [82]:
sue = {"children": 3,
"cats": 7,
"samoyeds": 2,
"pomeranians": 3,
"akitas": 0,
"vizslas": 0,
"goldfish": 5,
"trees": 3,
"cars": 2,
"perfumes": 1,
}

In [103]:
count = 0
for s in content:
    rslt = re.findall(r"Sue (\d+): (\w+): (\d+), (\w+): (\d+), (\w+): (\d+)", s)[0]
    numsue = int(rslt[0])
    keys = [r for r in rslt[1::2]]
    values = [int(r) for r in rslt[2::2]]
    
    bad = False
    for k, v in zip(keys, values):
        if sue[k] != v:
            bad = True
    
    if not bad:
        print(s)
        count += 1

Sue 213: children: 3, goldfish: 5, vizslas: 0


In [104]:
count = 0
for s in content:
    rslt = re.findall(r"Sue (\d+): (\w+): (\d+), (\w+): (\d+), (\w+): (\d+)", s)[0]
    numsue = int(rslt[0])
    keys = [r for r in rslt[1::2]]
    values = [int(r) for r in rslt[2::2]]
    
    bad = False
    for k, v in zip(keys, values):
        if k in ["cats", "trees"]:
            if v < sue[k]:
                bad = True
                break
        elif k in ["pomeranians", "goldfish"]:
            if v > sue[k]:
                bad = True
                break
        else:
            if sue[k] != v:
                bad = True
                break
    
    if not bad:
        print(s)
        count += 1

Sue 213: children: 3, goldfish: 5, vizslas: 0
Sue 323: perfumes: 1, trees: 6, goldfish: 0


# Day 17

In [132]:
x = np.vectorize(int)([11, 30, 47, 31, 32, 36, 3, 1, 5, 3, 32, 36, 15, 11, 46, 26, 28, 1, 19, 3])
c = 0

for i in range(1 << len(x)):
    t = i
    s = 0
    count = 0
    for j in x:
        if t % 2 == 1:
            s += j
            count += 1
            
        t //= 2
    
    if count == 4:
        if s == 150:
            c += 1
            
print(c)

4 9999


# Day 18

In [166]:
with open("data/adventofcode.com_2015_day_18_input") as fid:
    content = fid.read().replace('\n', "")

In [184]:
from collections import Counter

In [167]:
grid = np.zeros((100 * 100), dtype=int)
for i in range(len(content)):
    if content[i] == "#":
        grid[i] = 1
        
grid = grid.reshape((100, 100))

In [189]:
track = np.zeros((100, 102, 102), dtype=int)
padded_grid = np.zeros((102, 102), dtype=int)
padded_grid[1:-1, 1:-1] = grid.copy()
new_grid = padded_grid.copy()

for step in range(100):
    son = 0
    soff= 0
    new_grid = padded_grid.copy()
    for i in range(1, 101):
        for j in range(1, 101):
            n = padded_grid[i - 1 : i + 2, j - 1: j + 2].sum() - padded_grid[i, j]
            
            if padded_grid[i, j] == 1:
                if not (n == 2 or n == 3):
                    soff += 1
                    new_grid[i, j] = 0
            else:
                if n == 3:
                    son += 1
                    new_grid[i, j] = 1
                        
    track[step, :, :] = new_grid.copy()
    padded_grid = new_grid.copy()
    
    print(f"ON: {son}, OFF: {soff}, grid: {padded_grid.sum()}")

ON: 1102, OFF: 3298, grid: 2804
ON: 1276, OFF: 1459, grid: 2621
ON: 1185, OFF: 1259, grid: 2547
ON: 1130, OFF: 1186, grid: 2491
ON: 1042, OFF: 1227, grid: 2306
ON: 1081, OFF: 1062, grid: 2325
ON: 960, OFF: 1147, grid: 2138
ON: 933, OFF: 987, grid: 2084
ON: 858, OFF: 929, grid: 2013
ON: 861, OFF: 898, grid: 1976
ON: 827, OFF: 898, grid: 1905
ON: 828, OFF: 884, grid: 1849
ON: 773, OFF: 815, grid: 1807
ON: 786, OFF: 738, grid: 1855
ON: 728, OFF: 841, grid: 1742
ON: 698, OFF: 793, grid: 1647
ON: 710, OFF: 772, grid: 1585
ON: 673, OFF: 724, grid: 1534
ON: 663, OFF: 696, grid: 1501
ON: 638, OFF: 649, grid: 1490
ON: 627, OFF: 650, grid: 1467
ON: 628, OFF: 593, grid: 1502
ON: 602, OFF: 650, grid: 1454
ON: 556, OFF: 605, grid: 1405
ON: 564, OFF: 582, grid: 1387
ON: 553, OFF: 603, grid: 1337
ON: 504, OFF: 602, grid: 1239
ON: 508, OFF: 482, grid: 1265
ON: 525, OFF: 539, grid: 1251
ON: 501, OFF: 527, grid: 1225
ON: 495, OFF: 541, grid: 1179
ON: 486, OFF: 468, grid: 1197
ON: 496, OFF: 482, grid: 12

In [194]:
track = np.zeros((100, 102, 102), dtype=int)
padded_grid = np.zeros((102, 102), dtype=int)
padded_grid[1:-1, 1:-1] = grid.copy()
padded_grid[1, 1] = 1
padded_grid[1, -2] = 1
padded_grid[-2, 1] = 1
padded_grid[-2, -2] = 1
new_grid = padded_grid.copy()

for step in range(100):
    son = 0
    soff= 0
    new_grid = padded_grid.copy()
    for i in range(1, 101):
        for j in range(1, 101):
            if (i, j) in [(1, 1), (1, 100), (100, 1), (100, 100)]:
                continue
                
            n = padded_grid[i - 1 : i + 2, j - 1: j + 2].sum() - padded_grid[i, j]
            
            if padded_grid[i, j] == 1:
                if not (n == 2 or n == 3):
                    soff += 1
                    new_grid[i, j] = 0
            else:
                if n == 3:
                    son += 1
                    new_grid[i, j] = 1
                        
    track[step, :, :] = new_grid.copy()
    padded_grid = new_grid.copy()
    
    print(f"ON: {son}, OFF: {soff}, grid: {padded_grid.sum()}")

ON: 1101, OFF: 3299, grid: 2805
ON: 1274, OFF: 1459, grid: 2620
ON: 1184, OFF: 1257, grid: 2547
ON: 1130, OFF: 1186, grid: 2491
ON: 1043, OFF: 1231, grid: 2303
ON: 1080, OFF: 1061, grid: 2322
ON: 963, OFF: 1143, grid: 2142
ON: 929, OFF: 993, grid: 2078
ON: 850, OFF: 922, grid: 2006
ON: 852, OFF: 898, grid: 1960
ON: 820, OFF: 884, grid: 1896
ON: 814, OFF: 887, grid: 1823
ON: 761, OFF: 801, grid: 1783
ON: 770, OFF: 730, grid: 1823
ON: 729, OFF: 819, grid: 1733
ON: 695, OFF: 780, grid: 1648
ON: 714, OFF: 765, grid: 1597
ON: 680, OFF: 733, grid: 1544
ON: 661, OFF: 704, grid: 1501
ON: 636, OFF: 646, grid: 1491
ON: 621, OFF: 658, grid: 1454
ON: 624, OFF: 572, grid: 1506
ON: 581, OFF: 652, grid: 1435
ON: 542, OFF: 571, grid: 1406
ON: 565, OFF: 576, grid: 1395
ON: 551, OFF: 584, grid: 1362
ON: 511, OFF: 598, grid: 1275
ON: 512, OFF: 482, grid: 1305
ON: 535, OFF: 555, grid: 1285
ON: 514, OFF: 539, grid: 1260
ON: 509, OFF: 548, grid: 1221
ON: 506, OFF: 482, grid: 1245
ON: 520, OFF: 494, grid: 12

# Day 19

In [4]:
with open("data/adventofcode.com_2015_day_19_input.txt") as fid:
    content = fid.read()

In [95]:
def read(s):
    return [i.strip() for i in open(s, 'r')]
lines = read("data/adventofcode.com_2015_day_19_input.txt")

replacements = []
for i in lines[:-2]:
    m = re.findall(r'(\S+) => (\S+)', i)
    replacements.append(m[0])
X = lines[-1]

In [98]:
S = set()
for i, j in replacements:
    for k in range(len(X)):
        if X[k:k+len(i)] == i:
            y = X[:k] + j + X[k+len(i):]
            S.add(y)
len(S)

576

In [105]:
from random import shuffle

reps = replacements
mol = X

target = mol
part2 = 0

while target != 'e':
    tmp = target
    for a, b in reps:
        if b not in target:
            continue

        target = target.replace(b, a, 1)
        part2 += 1

    if tmp == target:
        target = mol
        part2 = 0
        shuffle(reps)

print(part2)

207


# Day 20

In [6]:
def algo20():
    nbpresent = 34000000
    
    house = np.zeros((34000000), dtype=np.int32)
    for i in range(1, nbpresent // 10):
        count = 0
        for j in range(i, nbpresent // 10, i):
            count += 1
            house[j] += i * 11
            if count > 50:
                break
            
    return house

In [7]:
h = algo20()

In [8]:
np.min(np.where(h >= 34000000)[0])

831600

# Day 21

In [90]:
from typing import Dict
from itertools import permutations, product

In [25]:
Weapons = {}
Weapons["Dagger"] = {"cost": 8, "dmg": 4, "armor": 0}
Weapons["Shortsword"] = {"cost": 10, "dmg": 5, "armor": 0}
Weapons["Warhammer"] = {"cost": 25, "dmg": 6, "armor": 0}
Weapons["Longsword"] = {"cost": 40, "dmg": 7, "armor": 0}
Weapons["Greataxe"] = {"cost": 74, "dmg": 8, "armor": 0}

Armor = {}
Armor["Leather"] = {"cost": 13, "dmg": 0, "armor": 1}
Armor["Chainmail"] = {"cost": 31, "dmg": 0, "armor": 2}
Armor["Splintmail"] = {"cost": 53, "dmg": 0, "armor": 3}
Armor["Bandedmail"] = {"cost": 75, "dmg": 0, "armor": 4}
Armor["Platemail"] = {"cost": 102, "dmg": 0, "armor": 5}

Rings = {}
Rings["Damage1"] = {"cost": 25, "dmg": 1, "armor": 0}
Rings["Damage2"] = {"cost": 50, "dmg": 2, "armor": 0}
Rings["Damage3"] = {"cost": 100, "dmg": 3, "armor": 0}
Rings["Defense1"] = {"cost": 20, "dmg": 0, "armor": 1}
Rings["Defense2"] = {"cost": 40, "dmg": 0, "armor": 2}
Rings["Defense3"] = {"cost": 80, "dmg": 0, "armor": 3}

In [102]:
class Character:
    def __init__(self, name, hp=100, attack=0, armor=0):
        self.name = name
        self.hp = hp
        self.attack = attack
        self.armor = armor
        self.gear = []
        self.gold_value = 0
        self.is_alive = True
        
    def set_armor(self, armor):
        self.armor = armor
        
    def set_attack(self, attack):
        self.attack = attack
        
    def set_equipment(self, name, equip: Dict):
        self.gold_value += equip["cost"]
        self.attack += equip["dmg"]
        self.armor += equip["armor"]
        self.gear.append(name)
#         self.print_stat()
        
    def get_attack(self):
        return self.attack
    
    def get_hit(self, char: Character):
        if not self.is_alive:
            raise ValueError("Game Over.")
            
        total = char.attack - self.armor
        if total < 1:
            total = 1
        self.hp -= total
#         print(f"{char.name} deals {char.attack}-{self.armor}={total} damage; {self.name} goes down to {self.hp} hit points")
        self.check_alive()
        
    def print_stat(self):
        print(f"{self.name} has {len(self.gear)} pieces of gear: {', '.join(self.gear)} for {self.attack} attack, {self.armor} armor and a value of {self.gold_value} gils.")
        
    def check_alive(self):
        if self.hp <= 0:
            print(f"{self.name} is dead!")
            self.is_alive = False
            

In [103]:
you = Character("Player", 100, 0, 0)
boss = Character("Boss", 109, 8, 2)

In [128]:
rings = [*permutations(Rings.keys(), 0)]
rings.extend([*permutations(Rings.keys(), 1)])
rings.extend([*permutations(Rings.keys(), 2)])
store = [*product(Weapons.keys(), ["", *Armor.keys()], rings)]

In [129]:
cost_win = []
cost_lose = []
losing_gear = []
for weap, armo, ring in store:
    you = Character("Player", 100, 0, 0)
    boss = Character("Boss", 109, 8, 2)
    if weap != '':
        you.set_equipment(weap, Weapons[weap])
    if armo != '':
        you.set_equipment(armo, Armor[armo])
    if len(ring) > 0:
        for r in ring:
            you.set_equipment(r, Rings[r])
            
    while True:
        boss.get_hit(you)
        if not boss.is_alive:
            cost_win.append(you.gold_value)
            break

        you.get_hit(boss)
        if not you.is_alive:
            cost_lose.append(you.gold_value)
            losing_gear.append((weap, armo, ring))
            break

Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Player is dead!
Boss is dead!
Player is dead!
Player is dead!
Player is dead!
Player is 

In [130]:
min(cost_win)

111

In [131]:
max(cost_lose)

188

In [132]:
losing_gear[np.argmax(cost_lose)]

('Dagger', '', ('Damage3', 'Defense3'))